### View image sequence with target bounding box and prediction

In [ ]:
import cv2
import os
import numpy as np

def load_timestamps(timestamp_file):
    """Load timestamps from the given file."""
    with open(timestamp_file, 'r') as f:
        timestamps = [line.strip() for line in f.readlines()]
    return timestamps

def load_labels(label_file):
    """Load bounding box labels from the given file."""
    labels = []
    with open(label_file, 'r') as f:
        for line in f.readlines():
            values = line.split()
            frame = int(values[0])
            track_id = int(values[1])
            obj_type = values[2]
            truncated = float(values[3])
            occluded = int(values[4])
            alpha = float(values[5])
            bbox = list(map(float, values[6:10]))  # left, top, right, bottom
            dimensions = list(map(float, values[10:13]))  # height, width, length
            location = list(map(float, values[13:16]))  # x, y, z in camera coordinates
            rotation_y = float(values[16])
            
            labels.append({
                'frame': frame,
                'track_id': track_id,
                'type': obj_type,
                'truncated': truncated,
                'occluded': occluded,
                'alpha': alpha,
                'bbox': bbox,
                'dimensions': dimensions,
                'location': location,
                'rotation_y': rotation_y
            })
    return labels

def draw_bounding_boxes(image, frame_labels):
    """Draw bounding boxes on the image."""
    for obj in frame_labels:
        left, top, right, bottom = map(int, obj['bbox'])
        obj_type = obj['type']
        
        # Set color based on object type
        if obj_type == 'Car':
            color = (0, 255, 0)  # Green
        elif obj_type == 'Pedestrian':
            color = (255, 0, 0)  # Blue
        elif obj_type == 'Cyclist':
            color = (0, 0, 255)  # Red
        else:
            color = (255, 255, 255)  # White

        # Draw rectangle and label
        cv2.rectangle(image, (left, top), (right, bottom), color, 2)
        cv2.putText(image, obj_type, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return image

def visualize_sequence(seq_folder):
    """Visualize the video sequence with bounding boxes."""
    # Load timestamps and labels
    image_folder = os.path.join(seq_folder, 'image_02\\data')
    timestamp_file = os.path.join(seq_folder, 'image_02\\timestamps.txt')
    label_file = os.path.join(seq_folder, 'labels.txt')

    timestamps = load_timestamps(timestamp_file)
    labels = load_labels(label_file)

    # Group labels by frame for quick access
    labels_by_frame = {}
    for label in labels:
        frame = label['frame']
        if frame not in labels_by_frame:
            labels_by_frame[frame] = []
        labels_by_frame[frame].append(label)

    # Get the list of images
    image_files = sorted(os.listdir(image_folder))

    for i, image_file in enumerate(image_files):
        frame_no = i  # frame number corresponds to the index of image file
        image_path = os.path.join(image_folder, image_file)
        image = cv2.imread(image_path)

        if image is None:
            print(f"Error reading image: {image_path}")
            continue

        # Check if there are labels for the current frame
        frame_labels = labels_by_frame.get(frame_no, [])
        if frame_labels:
            image = draw_bounding_boxes(image, frame_labels)

        # Display image with bounding boxes
        timestamp = timestamps[i] if i < len(timestamps) else 'N/A'
        cv2.putText(image, f"Timestamp: {timestamp}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        cv2.imshow('Sequence Viewer', image)

        # Press 'q' to exit
        if cv2.waitKey(200) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()

if __name__ == "__main__":
    # Set the root path to your project directory
    root_path = r'C:\Users\claes\Documents\Mine dokumenter\Universitet\7.semester\Perception for autonomous systems\Final project\34759_final_project_rect'

    # Outcomment my path and put in your own below:
    #root_path = r'C:\your_path'

    # Specify the sequence folder you want to visualize
    sequence_folder = os.path.join(root_path, 'seq_01')
    visualize_sequence(sequence_folder)

### Show if it is truncated or occluded

In [ ]:
import cv2
import os
import numpy as np

def load_timestamps(timestamp_file):
    """Load timestamps from the given file."""
    with open(timestamp_file, 'r') as f:
        timestamps = [line.strip() for line in f.readlines()]
    return timestamps

def load_labels(label_file):
    """Load bounding box labels from the given file."""
    labels = []
    with open(label_file, 'r') as f:
        for line in f.readlines():
            values = line.split()
            frame = int(values[0])
            track_id = int(values[1])
            obj_type = values[2]
            truncated = float(values[3])
            occluded = int(values[4])
            alpha = float(values[5])
            bbox = list(map(float, values[6:10]))  # left, top, right, bottom
            dimensions = list(map(float, values[10:13]))  # height, width, length
            location = list(map(float, values[13:16]))  # x, y, z in camera coordinates
            rotation_y = float(values[16])
            
            labels.append({
                'frame': frame,
                'track_id': track_id,
                'type': obj_type,
                'truncated': truncated,
                'occluded': occluded,
                'alpha': alpha,
                'bbox': bbox,
                'dimensions': dimensions,
                'location': location,
                'rotation_y': rotation_y
            })
    return labels

def draw_bounding_boxes(image, frame_labels):
    """Draw bounding boxes on the image with occlusion and truncation information."""
    for obj in frame_labels:
        left, top, right, bottom = map(int, obj['bbox'])
        truncated = obj['truncated']
        occluded = obj['occluded']

        # Set color based on occlusion level
        if occluded == 0:
            color = (0, 255, 0)    # Green for fully visible
        elif occluded == 1:
            color = (0, 255, 255)  # Yellow for partly occluded
        elif occluded == 2:
            color = (0, 165, 255)  # Orange for largely occluded
        else:
            color = (255, 255, 255)  # White for unknown occlusion

        # Draw rectangle
        cv2.rectangle(image, (left, top), (right, bottom), color, 2)

        # Create occlusion and truncation labels
        occlusion_text = f"Occluded: {occluded}"
        truncation_text = f"Truncated: {truncated:.2f}"

        # Draw text above the bounding box
        text_y = top - 10 if top - 10 > 10 else top + 10
        cv2.putText(image, occlusion_text, (left, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        cv2.putText(image, truncation_text, (left, text_y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    return image

def visualize_sequence(seq_folder):
    """Visualize the video sequence with bounding boxes."""
    # Load timestamps and labels
    image_folder = os.path.join(seq_folder, 'image_02\\data')
    timestamp_file = os.path.join(seq_folder, 'image_02\\timestamps.txt')
    label_file = os.path.join(seq_folder, 'labels.txt')

    timestamps = load_timestamps(timestamp_file)
    labels = load_labels(label_file)

    # Group labels by frame for quick access
    labels_by_frame = {}
    for label in labels:
        frame = label['frame']
        if frame not in labels_by_frame:
            labels_by_frame[frame] = []
        labels_by_frame[frame].append(label)

    # Get the list of images
    image_files = sorted(os.listdir(image_folder))

    for i, image_file in enumerate(image_files):
        frame_no = i  # frame number corresponds to the index of image file
        image_path = os.path.join(image_folder, image_file)
        image = cv2.imread(image_path)

        if image is None:
            print(f"Error reading image: {image_path}")
            continue

        # Check if there are labels for the current frame
        frame_labels = labels_by_frame.get(frame_no, [])
        if frame_labels:
            image = draw_bounding_boxes(image, frame_labels)

        # Display image with bounding boxes
        timestamp = timestamps[i] if i < len(timestamps) else 'N/A'
        cv2.putText(image, f"Timestamp: {timestamp}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        cv2.imshow('Sequence Viewer', image)

        # Press 'q' to exit
        if cv2.waitKey(200) & 0xFF == ord('q'):  # Adjusted frame speed to 100ms delay
            break

    cv2.destroyAllWindows()

if __name__ == "__main__":
    # Set the root path to your project directory
    root_path = r'C:\Users\claes\Documents\Mine dokumenter\Universitet\7.semester\Perception for autonomous systems\Final project\34759_final_project_rect'

    # Outcomment my path and put in your own below:
    #root_path = r'C:\your_path'
    
    # Specify the sequence folder you want to visualize
    sequence_folder = os.path.join(root_path, 'seq_02')
    visualize_sequence(sequence_folder)